In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn import svm, linear_model, ensemble
from sklearn import model_selection
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
from scipy.stats import randint, uniform
from pyswarm import pso

In [3]:
labels = pd.read_csv("antibacterial_lable.csv", index_col=0 )
data = pd.read_csv("mibig(antismash).csv", index_col=0 )

In [4]:
def get_sample_responses(vectors, responses):
    samples = vectors
    index = list(set(samples.index).intersection(set(labels.index)))
    responses = responses.loc[index]
    samples = samples.loc[index]
    return samples, responses

In [5]:
X , y = get_sample_responses(data, labels)
bgc_ids = pd.Series(y.index)

In [6]:
def objective(params):
    alpha, normalize, copy_X, class_weight, solver = params
    

    normalize =[False, True][int(normalize)]
    copy_X = [False, True][int(copy_X)]
    class_weight = ['balanced',None][int(class_weight)]
    solver = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'][int(solver)]
    
    clf = linear_model.RidgeClassifier(
        alpha=alpha,
        normalize=normalize,
        copy_X=copy_X,
        class_weight=class_weight,
        solver=solver)
    
    acc_kf = []
    
    cv = KFold(n_splits=5,random_state=0,shuffle=True)
    for i, (id_train, id_val) in enumerate(cv.split(bgc_ids)):
        train_ids, val_ids = bgc_ids[id_train], bgc_ids[id_val]
        X_train, X_val = X.loc[train_ids].values, X.loc[val_ids].values
        y_train, y_val = y.loc[train_ids].values, y.loc[val_ids].values
        #print('Train:', len(X_train), 'Test:', len(X_val))
    
        clf.fit(X_train, y_train)
        pred = clf.predict(X_val)
    
        acc = accuracy_score(pred, y_val)
        acc_kf.append(acc)
        
    acc_kf = np.array(acc_kf).mean()
    
    return -acc_kf

In [7]:
lb = [0.1, 0, 0, 0, 0] 
ub = [10.0, 1, 1, 1, 6]

In [8]:
%%time
best_params, best_score = pso(objective, lb, ub, swarmsize=50, maxiter=1)

Stopping search: maximum iterations reached --> 1
CPU times: user 30min 40s, sys: 27min 28s, total: 58min 8s
Wall time: 13min 27s
